<a href="https://colab.research.google.com/github/Tiabet/Complete_story/blob/master/KoGPT_Finetuning_without_Eval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
%%capture
!pip install transformers
!pip install accelerate
!pip install datasets
!pip install evaluate
!pip install rouge

In [5]:
import torch
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel, DataCollatorWithPadding,TrainingArguments, Trainer, GPT2Config, TrainerCallback
from datasets import Dataset, load_dataset
from rouge import Rouge
import os
import numpy as np

모델 호출

In [6]:
model_checkpoint = "skt/kogpt2-base-v2"

tokenizer = PreTrainedTokenizerFast.from_pretrained(model_checkpoint, bos_token='<bos>', eos_token='<eos>', unk_token='<unk>',
  pad_token='<pad>', mask_token='<mask>', sep_token = '<sep>', padding_side='left')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
# Load the existing model checkpoint
model_config = GPT2Config.from_pretrained(model_checkpoint)

# Add the new special token to the model's config
model_config.bos_token_id = tokenizer.bos_token_id
model_config.eos_token_id = tokenizer.eos_token_id
model_config.unk_token_id = tokenizer.unk_token_id
model_config.pad_token_id = tokenizer.pad_token_id
model_config.mask_token_id = tokenizer.mask_token_id
model_config.sep_token_id = tokenizer.sep_token_id  # Add this line

# Initialize the model with the updated configuration
model = GPT2LMHeadModel.from_pretrained(model_checkpoint, config=model_config)

model.resize_token_embeddings(len(tokenizer))

You are resizing the embedding layer without providing a `pad_to_multiple_of` parameter. This means that the new embedding dimension will be 51203. This might induce some performance reduction as *Tensor Cores* will not be available. For more details about this, or help on choosing the correct value for resizing, refer to this guide: https://docs.nvidia.com/deeplearning/performance/dl-performance-matrix-multiplication/index.html#requirements-tc


Embedding(51203, 768)

데이터셋 구축

In [10]:
dataset = load_dataset('drive/MyDrive', data_files={
    'train': 'nikluge-sc-2023-train.jsonl',
    'validation': 'nikluge-sc-2023-dev.jsonl',
    'test': 'nikluge-sc-2023-test.jsonl'
})

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [12]:
sentence1 = []
sentence3 = []

for dictionary in dataset['train']['input']:
    sentence1.append(dictionary["sentence1"])
    sentence3.append(dictionary["sentence3"])

dataset['train'] = dataset['train'].add_column('sentence1', sentence1)
dataset['train'] = dataset['train'].add_column('sentence3', sentence3)

In [13]:
sentence1 = []
sentence3 = []

for dictionary in dataset['validation']['input']:
    sentence1.append(dictionary["sentence1"])
    sentence3.append(dictionary["sentence3"])

dataset['validation'] = dataset['validation'].add_column('sentence1', sentence1)
dataset['validation'] = dataset['validation'].add_column('sentence3', sentence3)

In [14]:
sentence1 = []
sentence3 = []

for dictionary in dataset['test']['input']:
    sentence1.append(dictionary["sentence1"])
    sentence3.append(dictionary["sentence3"])

dataset['test'] = dataset['test'].add_column('sentence1', sentence1)
dataset['test'] = dataset['test'].add_column('sentence3', sentence3)

In [27]:
def tokenize_function(batch):
    sentences1 = batch['sentence1']
    sentences3 = batch['sentence3']
    outputs = batch['output']

    model_inputs = []

    for s1, s3, output in zip(sentences1, sentences3, outputs):
        # Convert list elements to strings
        sentence1_str = ''.join(map(str, s1))
        sentence3_str = ''.join(map(str, s3))
        output_str = ''.join(map(str,output))

        # Tokenize the input sentences
        inputs = tokenizer(
            '<bos>' + sentence1_str + '<sep>' + sentence3_str +\
            '<sep>' + output_str + '<eos>',
            padding="max_length",
            max_length=128,  # Adjust this as needed
            truncation=True,
            return_attention_mask = True,
            return_tensors = 'pt'
        )

        # Append tokenized inputs and labels to the lists
        model_inputs.append(inputs)

    return {
        'input_ids': [item['input_ids'] for item in model_inputs],
        'attention_mask': [item['attention_mask'] for item in model_inputs],
        'labels': [item['input_ids'] for item in model_inputs],
    }

In [28]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/120140 [00:00<?, ? examples/s]

Map:   0%|          | 0/15017 [00:00<?, ? examples/s]

Map:   0%|          | 0/15018 [00:00<?, ? examples/s]

In [29]:
tokenized_datasets = tokenized_datasets.remove_columns(dataset["train"].column_names)

In [30]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 120140
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 15017
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 15018
    })
})